by mingyu

타겟 변경

- t+1 ~ t+14 까지는 (t+n - t)/t 로 계산

ans_1_14.append((globals()[f'set_df_{k}'][c].iloc[i+1]- base_d)/base_d)  

ans_1_14.append((globals()[f'set_df_{k}'][c].iloc[i+1])/base_d)

- t+22 ~ t+28은 np.mean(t+22 ~ t+28) - t / t

ans_22_28 = (globals()[f'set_df_{k}'][c][22:29].mean() - base_d)/base_d 

ans_22_28 = (globals()[f'set_df_{k}'][c][22:29].mean())/base_d

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
from glob2 import glob
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from pandasql import sqldf
import random
import os
from numba import jit

test_num = 9

# 경고 끄기
warnings.filterwarnings(action='ignore')

# 시드고정
tf.random.set_seed(19970119)
random.seed(19970119)
np.random.seed(19970119)

In [14]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


&nbsp;

## 입력 shape 및 형태 정의 함수

In [2]:
@jit
def make_Tensor(array):
    return tf.convert_to_tensor(array, dtype=tf.float32)

@jit
def astype_data(data):
    df = data.astype(np.float32)
    return make_Tensor(df)

&nbsp;

## Transformer 정의

- encoder

In [3]:
@jit
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

- build

In [4]:
@jit
def build_model(input_shape, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout=0, mlp_dropout=0):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(28)(x) # 4주 예측
    return keras.Model(inputs, outputs)

&nbsp;

## keras eraly stop, chekpoint 정의

In [5]:
@jit
def call_back_set(name, epoch, batch_size):
    early_stopping = EarlyStopping(monitor='val_loss', patience=100)

    if os.path.exists(f'./check') == False:
        os.mkdir(f'./check')

    filename = f'./check/{name}-{epoch}-{batch_size}.h5'

    checkpoint = ModelCheckpoint(filename,
                                 monitor='val_loss',
                                 verbose=0,
                                 save_best_only=True,
                                 save_weights_only=True,
                                 mode='auto'
                                 )
    return [early_stopping, checkpoint]

&nbsp;

## Model 훈련 함수

In [15]:
def train(x_train, y_train, x_val, y_val, name, epoch, batch_size, learning_rate = 0.0015, verbose = 1):
    model = build_model(
    x_train.shape[1:],
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=3,
    mlp_units=[128],
    mlp_dropout=0.3,
    dropout=0.3,
    )

    model.compile(
        loss="mean_squared_error",
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate)
    )

    # Train the model
    with tf.device('/device:GPU: 1'):
        history1 = model.fit(
            x_train, y_train,
            epochs = epoch,
            steps_per_epoch=len(x_train) / batch_size,
            batch_size=batch_size,
            validation_data=(x_val, y_val),
            validation_steps=len(x_val) / batch_size,
            shuffle=False,
            callbacks=call_back_set(name, epoch, batch_size),
            verbose=verbose)

    return model

&nbsp;

## 시점 윈도우 생성 함수

In [16]:
@jit
def time_window(df, t, t_sep):
    seq_len = t
    seqence_length = seq_len + t_sep

    result = []
    for index in range(len(df) - seqence_length):
        result.append(df[index: index + seqence_length].values)

    return np.array(result)

@jit
def y_time_window(df, t, t_sep):
    seq_len = t
    seqence_length = seq_len + t_sep

    result = []
    for index in range(14, len(df) - seqence_length):
        result.append(df[index: index + seqence_length].values - df.iloc[index-1])

    return np.array(result)

&nbsp;

## 데이터 불러오기 및 parameter 설정

In [17]:
data_list = glob('./data//train/*.csv')
epoch = 5000
batch = 64
tr_del_list = ['단가(원)', '거래량', '거래대금(원)', '경매건수', '도매시장코드', '도매법인코드', '산지코드 '] # train 에서 사용하지 않는 열
ts_del_list = ['단가(원)', '거래량', '거래대금(원)', '경매건수', '도매시장코드', '도매법인코드', '산지코드 ', '해당일자_전체평균가격(원)'] # test 에서 사용하지 않는 열
check_col = ['일자구분_중순', '일자구분_초순', '일자구분_하순','월구분_10월', '월구분_11월', '월구분_12월', '월구분_1월', '월구분_2월', '월구분_3월', 
             '월구분_4월','월구분_5월', '월구분_6월', '월구분_7월', '월구분_8월', '월구분_9월'] # 열 개수 맞추기

&nbsp;

## Train 과정

In [18]:
from sklearn.preprocessing import MinMaxScaler

losses = []
scalers = {}

for i in tqdm(data_list):
    df_number = i.split("_")[-1].split(".")[0]
    df = pd.read_csv(i)

    for j in df.columns:
        df[j] = df[j].replace({' ': np.nan})

    # 사용할 열 선택 및 index 설정
    df.drop(tr_del_list, axis=1, inplace=True)
    df.set_index('datadate', drop=True, inplace=True)
    
    # nan 처리
    df = df.fillna(0)
    # df = df.interpolate(method = 'values')
    
    # 변수와 타겟 분리
    x, y = df[[i for i in df.columns if i != '해당일자_전체평균가격(원)']], df['해당일자_전체평균가격(원)']
    scaler = MinMaxScaler()
    x = pd.DataFrame(scaler.fit_transform(x))
    scalers[df_number] = scaler

    # 2주 입력을 통한 이후 4주 예측을 위해 y의 첫 14일을 제외
    # y = y[14:]

    # time series window 생성
    data_x = time_window(x, 13, 1)
    data_y = y_time_window(y, 27, 1)

    # y의 길이와 같은 길이로 설정
    xdata = data_x[:len(data_y)]
    ydata = data_y
    
    # train, validation 분리 (8 : 2)
    x_train, x_val, y_train, y_val = train_test_split(xdata, ydata, test_size=0.2, shuffle=False, random_state=119)

    # transformer 모델 훈련
    transformer = train(astype_data(x_train), y_train, astype_data(x_val), y_val, f'transformer-{df_number}', epoch,
                        batch, learning_rate = 0.001, verbose = 0)
    
    transformer.load_weights(f'./check/transformer-{df_number}-{epoch}-{batch}.h5')
    loss = transformer.evaluate(astype_data(x_val), y_val, verbose=0)
    print('\nvalid_loss: ', loss)
    losses.append(loss)

    if os.path.exists(f'./model') == False:
        os.mkdir(f'./model')

    # 모델 저장
    transformer.save(f'./model/transformer-{df_number}-{epoch}-{batch}.h5')

0it [00:00, ?it/s]


In [10]:
losses

[]

In [11]:
# [12726371.0,
#  5549861.0,
#  245057.046875,
#  838909.6875,
#  1330815.875,
#  56871424.0,
#  652660.1875,
#  5362711.5,
#  847211.25,
#  979644.75,
#  5531605.5,
#  277393.5625,
#  2626831.0,
#  1843104.75,
#  13118171.0,
#  21230410.0,
#  5809379.5,
#  170681344.0,
#  867232.5625,
#  1841623.125,
#  1510910.5,
#  83323736.0,
#  5779053.5,
#  242149.546875,
#  5717967.5,
#  2376440.0,
#  2871122.75,
#  7657346.5,
#  643398.5625,
#  5717721.0,
#  2783284.75,
#  506001.46875,
#  625200.0,
#  1445973.25,
#  19559360.0,
#  6589400.0,
#  4889610.5]

&nbsp;

## Test 과정

In [12]:
zero_csv = [0 for i in range(14)]  # 시점이 비어있는 데이터 0으로 채우기 위한 변수

for i in tqdm(range(10)):
    data_list = glob(f'./data/test/set_{i}/*.csv')
    for idx,j in enumerate(data_list):
        df = pd.read_csv(j)
        
        if len(df) == 0:
            df['zero_non'] = zero_csv
            df = df.fillna(0)
            df.drop('zero_non', axis=1, inplace=True)


        file_number = j.split('test_')[1].split('.')[0]

        # 사용할 열 선택, index 설정
        df.drop(ts_del_list, axis=1, inplace=True)
        df.set_index('datadate', drop=True, inplace=True)

        # train input 과 형상 맞추기
        add_col = [i for i in check_col if i not in df.columns]

        for a in add_col:
            df[a] = 0

        # ' ' -> nan 으로 변경
        for a in df.columns:
            df[a] = df[a].replace({' ': np.nan})

        # nan 처리
        df = df.fillna(0)
        # df = df.interpolate(method = 'values')

        df = pd.DataFrame(scalers[file_number].transform(df))

        # x_test  생성
        df_test = astype_data(df.values.reshape(1, df.values.shape[0], df.values.shape[1]))

        # model test
        if os.path.exists('./model_output') == False:
            os.mkdir('./model_output')

        if os.path.exists(f'./model_output/set_{i}') == False:
            os.mkdir(f'./model_output/set_{i}')

        # 해당하는 모델 불러오기
        model_test = tf.keras.models.load_model(f'./model/transformer-{file_number}-{epoch}-{batch}.h5')
        pred = model_test.predict(df_test)


        # 결과 저장
        save_df = pd.DataFrame(pred).T
        save_df.to_csv(f'./model_output/set_{i}/predict_{file_number}.csv', index=False)

100%|██████████| 10/10 [00:00<00:00, 48321.47it/s]


&nbsp;

## 정답 제출 파일생성

In [13]:
for k in tqdm(range(10)):
  globals()[f'set_df_{k}'] = pd.DataFrame()
  answer_df_list = glob(f'./model_output/set_{k}/*.csv') # 예측한 결과 불러오기
  pum_list = glob(f'./aT_test_raw/sep_{k}/*.csv') # 기존 test input 불러오기
  pummok = [a for a in pum_list if 'pummok' in a.split('/')[-1]]

  for i in answer_df_list:
    df = pd.read_csv(i)
    number = i.split('_')[-1].split('.')[0]

    base_number = 0
    for p in pummok:
      if number == p.split('_')[-1].split('.')[0]:
        pum_df = pd.read_csv(p)

        if len(pum_df) != 0:
           base_number = pum_df.iloc[len(pum_df)-1]['해당일자_전체평균가격(원)']  # 기존 각 sep 마다 test input의 마지막 target 값 가져오기 (변동률 계산을 위해)
        else:
          base_number = np.nan

    globals()[f'set_df_{k}'][f'품목{number}']  = [base_number] + list(df[df.columns[-1]].values) # 각 품목당 순서를 t, t+1 ... t+28 로 변경

  globals()[f'set_df_{k}'] = globals()[f'set_df_{k}'][[f'품목{col}' for col in range(37)]] # 열 순서를 품목0 ~ 품목36 으로 변경

  0%|          | 0/10 [00:00<?, ?it/s]


KeyError: "None of [Index(['품목0', '품목1', '품목2', '품목3', '품목4', '품목5', '품목6', '품목7', '품목8', '품목9',\n       '품목10', '품목11', '품목12', '품목13', '품목14', '품목15', '품목16', '품목17', '품목18',\n       '품목19', '품목20', '품목21', '품목22', '품목23', '품목24', '품목25', '품목26', '품목27',\n       '품목28', '품목29', '품목30', '품목31', '품목32', '품목33', '품목34', '품목35', '품목36'],\n      dtype='object')] are in the [columns]"

- 변동률 계산을 위한 t, t+1 ... t+28 설정

In [ ]:
set_df_0

,품목0,품목1,품목2,품목3,품목4,품목5,품목6,품목7,품목8,품목9,...,품목27,품목28,품목29,품목30,품목31,품목32,품목33,품목34,품목35,품목36
0,3871.125000,1362.117613,2909.783785,3400.075583,3947.809169,9253.947514,2717.280000,3361.030923,4911.899864,1173.018633,...,8640.811309,602.005658,1105.412623,1566.274239,3633.464557,5454.710444,5619.188362,5230.620027,2905.100888,2087.675036
1,-12.409256,-7.358492,-14.112641,-12.312008,-12.882431,-15.392607,-0.855508,-12.561447,-16.173920,-11.759997,...,-16.982496,-6.225435,-8.617882,-13.010770,0.022401,-16.104780,-10.537136,-14.239921,-11.320800,-12.931734
2,-8.863336,-8.081900,-13.895362,-13.483604,-13.872280,-12.380589,-0.562721,-11.105879,-15.953316,-13.746824,...,-9.743759,-7.954165,-9.679001,-15.436919,-0.219967,-11.773530,-6.140578,-9.772795,-8.813186,-9.731467
3,-11.841292,-7.894259,-16.069698,-14.460712,-15.613992,-12.336391,-0.577877,-11.187726,-18.407750,-16.835619,...,-10.607600,-8.785923,-11.382174,-17.492855,0.136916,-13.684232,-5.403549,-9.143981,-10.597400,-10.421328
4,-6.608344,-4.035626,-7.448795,-7.920938,-8.526505,-7.134778,-0.524427,-4.981292,-12.004331,-9.366172,...,-4.581506,-5.808335,-6.003347,-8.063651,-0.226550,-13.222817,-0.570843,-3.024974,-5.702443,-1.174031
5,0.373810,-0.714139,-0.290081,-3.801894,-3.518338,-2.860149,-0.609569,-0.286861,-3.928932,-0.822785,...,-1.740134,-3.248748,-1.797190,0.163888,0.206859,-10.384373,3.643076,0.719832,-2.032467,4.845383
6,2.334526,-0.367362,4.285327,-3.050652,-2.467442,-1.574163,-0.463092,2.794760,-0.796475,1.613108,...,-0.144577,-2.697415,-0.955542,2.877995,-0.041498,-7.878797,7.080860,1.679001,-0.692863,6.943962
7,1.529339,-0.811718,0.362376,-2.435102,-2.364306,-2.474210,-0.669188,6.321044,-3.067411,-1.820461,...,-0.787812,-3.052811,-2.143257,1.740214,0.032485,-8.408282,6.894304,1.575308,-1.951459,5.336755
8,-4.267825,-3.087603,-9.507582,-6.246930,-7.627026,-8.773832,-0.703125,1.750452,-12.245646,-10.648567,...,-5.944548,-5.982255,-7.540664,-7.068624,-0.300222,-16.361744,-0.683251,-2.739526,-6.625472,-2.370988
9,-4.166376,-3.548567,-9.114099,-7.633295,-9.302834,-7.532399,-0.520220,0.406256,-14.178009,-12.384176,...,-6.921358,-7.339405,-8.100257,-8.732373,-0.384718,-13.902085,1.400613,-1.339208,-7.186546,-2.331357


- 변동률 계산 

In [ ]:
date = [f'd+{i}' for i in range(1,15)] + ['d+22 ~ 28 평균']


for k in range(10):
  globals()[f'answer_df_{k}'] = pd.DataFrame()
  for c in globals()[f'set_df_{k}'].columns:
    base_d = globals()[f'set_df_{k}'][c][0] # 변동률 기준 t 값

    ans_1_14 = []
    for i in range(14):
      # ans_1_14.append((globals()[f'set_df_{k}'][c].iloc[i+1]- base_d)/base_d)  # t+1 ~ t+14 까지는 (t+n - t)/t 로 계산
      ans_1_14.append((globals()[f'set_df_{k}'][c].iloc[i+1])/base_d)

    # ans_22_28 = (globals()[f'set_df_{k}'][c][22:29].mean() - base_d)/base_d # t+22 ~ t+28은 np.mean(t+22 ~ t+28) - t / t
    ans_22_28 = (globals()[f'set_df_{k}'][c][22:29].mean())/base_d
    globals()[f'answer_df_{k}'][f'{c} 변동률'] = ans_1_14 + [ans_22_28]
  
  globals()[f'answer_df_{k}']['Set'] = k # set 번호 설정
  globals()[f'answer_df_{k}']['일자'] = date # 일자 설정

- sep 0  ~ sep 9 까지 합치기

In [ ]:
# 위에서 계산된 변동률 들을 합쳐주는 과정

all_df =pd.DataFrame()
for i in range(10):
  if i== 0 :
    all_df = pd.concat([all_df, globals()[f'answer_df_{i}']],axis=1)
  else:
    all_df = pd.concat([all_df, globals()[f'answer_df_{i}']])


all_df = all_df[['Set','일자'] + list(all_df.columns[:-2])]
all_df.reset_index(drop=True, inplace=True)

In [ ]:
answer_df_0

,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,품목8 변동률,품목9 변동률,...,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률,Set,일자
0,-0.003206,-0.005402,-0.004850,-0.003621,-0.003263,-0.001663,-0.000315,-0.003737,-0.003293,-0.010025,...,-0.007796,-0.008307,0.000006,-0.002952,-0.001875,-0.002722,-0.003897,-0.006194,0,d+1
1,-0.002290,-0.005933,-0.004775,-0.003966,-0.003514,-0.001338,-0.000207,-0.003304,-0.003248,-0.011719,...,-0.008756,-0.009856,-0.000061,-0.002158,-0.001093,-0.001868,-0.003034,-0.004661,0,d+2
2,-0.003059,-0.005796,-0.005523,-0.004253,-0.003955,-0.001333,-0.000213,-0.003329,-0.003748,-0.014352,...,-0.010297,-0.011168,0.000038,-0.002509,-0.000962,-0.001748,-0.003648,-0.004992,0,d+3
3,-0.001707,-0.002963,-0.002560,-0.002330,-0.002160,-0.000771,-0.000193,-0.001482,-0.002444,-0.007985,...,-0.005431,-0.005148,-0.000062,-0.002424,-0.000102,-0.000578,-0.001963,-0.000562,0,d+4
4,0.000097,-0.000524,-0.000100,-0.001118,-0.000891,-0.000309,-0.000224,-0.000085,-0.000800,-0.000701,...,-0.001626,0.000105,0.000057,-0.001904,0.000648,0.000138,-0.000700,0.002321,0,d+5
5,0.000603,-0.000270,0.001473,-0.000897,-0.000625,-0.000170,-0.000170,0.000832,-0.000162,0.001375,...,-0.000864,0.001837,-0.000011,-0.001444,0.001260,0.000321,-0.000238,0.003326,0,d+6
6,0.000395,-0.000596,0.000125,-0.000716,-0.000599,-0.000267,-0.000246,0.001881,-0.000624,-0.001552,...,-0.001939,0.001111,0.000009,-0.001541,0.001227,0.000301,-0.000672,0.002556,0,d+7
7,-0.001102,-0.002267,-0.003267,-0.001837,-0.001932,-0.000948,-0.000259,0.000521,-0.002493,-0.009078,...,-0.006822,-0.004513,-0.000083,-0.003000,-0.000122,-0.000524,-0.002281,-0.001136,0,d+8
8,-0.001076,-0.002605,-0.003132,-0.002245,-0.002356,-0.000814,-0.000191,0.000121,-0.002886,-0.010558,...,-0.007328,-0.005575,-0.000106,-0.002549,0.000249,-0.000256,-0.002474,-0.001117,0,d+9
9,-0.001616,-0.002862,-0.003796,-0.002461,-0.002576,-0.000821,-0.000248,0.000149,-0.003528,-0.012457,...,-0.008272,-0.006614,0.000083,-0.002788,0.000374,-0.000138,-0.002683,-0.001586,0,d+10


- 정답 양식으로 변경

In [ ]:
# set, 일자 기억하기위해 따로 저장

re_set = list(all_df['Set'])
re_date = list(all_df['일자'])


# 정답 양식 불러오기
out_ans = pd.read_csv('./answer_example.csv')

# 두 dataframe 합치기 (nan + 숫자 = nan 이용)
submit_df = all_df + out_ans

submit_df['Set'] = re_set
submit_df['일자'] = re_date


# 최종 저장
submit_df.to_csv('./submit.csv',index=False)

In [ ]:
# set, 일자 기억하기위해 따로 저장

re_set = list(all_df['Set'])
re_date = list(all_df['일자'])


# 정답 양식 불러오기
out_ans = pd.read_csv('./answer_example.csv')

# 두 dataframe 합치기 (nan + 숫자 = nan 이용)
submit_df = all_df + out_ans

submit_df['Set'] = re_set
submit_df['일자'] = re_date


# 최종 저장
submit_df.to_csv('../submit.csv',index=False)

- 계산된 변동률 결과물

In [ ]:
all_df

,Set,일자,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,...,품목27 변동률,품목28 변동률,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률
0,0,d+1,-0.003206,-0.005402,-0.004850,-0.003621,-0.003263,-0.001663,-0.000315,-0.003737,...,-0.001965,-0.010341,-0.007796,-0.008307,0.000006,-0.002952,-0.001875,-0.002722,-0.003897,-0.006194
1,0,d+2,-0.002290,-0.005933,-0.004775,-0.003966,-0.003514,-0.001338,-0.000207,-0.003304,...,-0.001128,-0.013213,-0.008756,-0.009856,-0.000061,-0.002158,-0.001093,-0.001868,-0.003034,-0.004661
2,0,d+3,-0.003059,-0.005796,-0.005523,-0.004253,-0.003955,-0.001333,-0.000213,-0.003329,...,-0.001228,-0.014594,-0.010297,-0.011168,0.000038,-0.002509,-0.000962,-0.001748,-0.003648,-0.004992
3,0,d+4,-0.001707,-0.002963,-0.002560,-0.002330,-0.002160,-0.000771,-0.000193,-0.001482,...,-0.000530,-0.009648,-0.005431,-0.005148,-0.000062,-0.002424,-0.000102,-0.000578,-0.001963,-0.000562
4,0,d+5,0.000097,-0.000524,-0.000100,-0.001118,-0.000891,-0.000309,-0.000224,-0.000085,...,-0.000201,-0.005397,-0.001626,0.000105,0.000057,-0.001904,0.000648,0.000138,-0.000700,0.002321
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,9,d+11,0.004448,-0.001149,0.000201,-0.007728,-0.010854,-0.000507,-0.000964,NaN,...,-0.000647,-0.008531,0.002618,0.002124,-0.000033,-0.002641,0.018524,-0.083729,-0.002258,0.002757
146,9,d+12,0.005481,-0.000332,0.002697,-0.008349,-0.014100,-0.000283,-0.000591,NaN,...,-0.000285,-0.006577,0.011998,0.005407,-0.000096,-0.001297,0.020676,-0.065204,-0.001372,0.004683
147,9,d+13,0.005577,-0.000620,0.002234,-0.007376,-0.013782,-0.000197,-0.000773,NaN,...,-0.000190,-0.007189,0.009053,0.004735,0.000035,-0.001130,0.024579,-0.025186,-0.001361,0.004455
148,9,d+14,0.006801,-0.000703,0.001179,-0.003990,-0.009245,-0.000055,-0.000640,NaN,...,-0.000030,-0.006664,0.001331,0.004689,-0.000002,-0.001266,0.027517,0.000917,-0.001122,0.004550


- 제출 양식

In [ ]:
out_ans

,Set,일자,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,...,품목27 변동률,품목28 변동률,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률
0,0,d+1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,d+2,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,d+3,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,d+4,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,d+5,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,9,d+11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
146,9,d+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
147,9,d+13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
148,9,d+14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


- 제출 양식 반영한 최종 결과물 (**실 제출용**)

In [ ]:
submit_df

,Set,일자,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,...,품목27 변동률,품목28 변동률,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률
0,0,d+1,-0.003206,-0.005402,-0.004850,-0.003621,-0.003263,-0.001663,-0.000315,-0.003737,...,-0.001965,-0.010341,-0.007796,-0.008307,0.000006,-0.002952,-0.001875,-0.002722,-0.003897,-0.006194
1,0,d+2,-0.002290,-0.005933,-0.004775,-0.003966,-0.003514,-0.001338,NaN,-0.003304,...,-0.001128,-0.013213,-0.008756,-0.009856,-0.000061,-0.002158,-0.001093,-0.001868,-0.003034,-0.004661
2,0,d+3,NaN,NaN,NaN,NaN,NaN,NaN,-0.000213,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,d+4,-0.001707,-0.002963,-0.002560,-0.002330,-0.002160,-0.000771,NaN,-0.001482,...,-0.000530,-0.009648,-0.005431,-0.005148,-0.000062,-0.002424,-0.000102,-0.000578,-0.001963,-0.000562
4,0,d+5,0.000097,-0.000524,-0.000100,-0.001118,-0.000891,-0.000309,NaN,-0.000085,...,-0.000201,-0.005397,-0.001626,0.000105,0.000057,-0.001904,0.000648,0.000138,-0.000700,0.002321
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,9,d+11,0.004448,-0.001149,0.000201,-0.007728,-0.010854,-0.000507,-0.000964,NaN,...,-0.000647,-0.008531,0.002618,0.002124,-0.000033,-0.002641,0.018524,-0.083729,-0.002258,0.002757
146,9,d+12,0.005481,-0.000332,0.002697,-0.008349,-0.014100,-0.000283,-0.000591,NaN,...,-0.000285,-0.006577,0.011998,0.005407,-0.000096,-0.001297,0.020676,-0.065204,-0.001372,0.004683
147,9,d+13,0.005577,-0.000620,0.002234,-0.007376,-0.013782,-0.000197,-0.000773,NaN,...,-0.000190,-0.007189,0.009053,0.004735,0.000035,-0.001130,0.024579,-0.025186,-0.001361,0.004455
148,9,d+14,0.006801,-0.000703,0.001179,-0.003990,-0.009245,-0.000055,-0.000640,NaN,...,-0.000030,-0.006664,0.001331,0.004689,-0.000002,-0.001266,0.027517,0.000917,-0.001122,0.004550
